# Facial Recognition Deep Learning

### Installing Dependencies

In [1]:
!pip install opencv-python matplotlib tensorflow-macos

#tensorflow~=2.6.0 and tensorflow-gpu==2.6.0 for non mac users

  Using cached opencv_python-4.7.0.68-cp37-abi3-macosx_11_0_arm64.whl (31.1 MB)
  Using cached matplotlib-3.6.3-cp39-cp39-macosx_11_0_arm64.whl (7.2 MB)
  Using cached Pillow-9.4.0-cp39-cp39-macosx_11_0_arm64.whl (3.0 MB)
  Using cached contourpy-1.0.7-cp39-cp39-macosx_11_0_arm64.whl (229 kB)
  Using cached fonttools-4.38.0-py3-none-any.whl (965 kB)
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)
  Using cached kiwisolver-1.4.4-cp39-cp39-macosx_11_0_arm64.whl (63 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)


## Importing Libraries

In [5]:
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten

In [6]:
# Input & Output Model Look Like
#Model(input=[inputimgage, verificationImage], output=[1,0])